In [1]:
import time
import pyspark
from pyspark.sql import SparkSession


BUCKET_INPUT_PATH = "gs://iasd-input-data"
DATASET_PATHS = {
    "notre_dame": f"{BUCKET_INPUT_PATH}/web-NotreDame.txt",
    "berk_stan.txt": f"{BUCKET_INPUT_PATH}/web-BerkStan.txt",
    "stanford.txt": f"{BUCKET_INPUT_PATH}/web-Stanford.txt",
    "google.txt": f"{BUCKET_INPUT_PATH}/web-Google.txt"
}

dataset_path = f"{BUCKET_INPUT_PATH}/test.txt"

In [2]:
spark_session = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

spark_context = spark_session.sparkContext

spark_session

In [3]:
def load_rdd(path):
    return spark_context.textFile(path)


def preprocess_rdd(rdd):
    return rdd.filter(lambda x: "#" not in x) \
                .map(lambda x: x.split("\t")) \
                .map(lambda x: (int(x[0]), int(x[1])))

In [ ]:
rdd_raw = load_rdd(dataset_path)
rdd_raw.take(5)

In [ ]:
rdd = preprocess_rdd(rdd_raw)
rdd.take(5)

iterate map

In [4]:
def iterate_map_rdd(rdd):
    return rdd.union(rdd.map(lambda x : (x[1], x[0])))

In [ ]:
rdd_iterate_map = iterate_map_rdd(rdd)
rdd_iterate_map.take(10)

In [5]:
# good
new_pairs = spark_context.accumulator(0)
# countnew_pairs function to know if additional CCF iteration is needed
def count_new_pairs(x):
  global new_pairs
  k, values = x
  min, value_list = k, []
  for v in values:
    if v < min:
       min = v
    value_list.append(v)
  if min < k:
    yield((k, min))
    for v in value_list:
      if min != v:
        new_pairs += 1
        yield((v, min))
        

def iterate_reduce_rdd(rdd):
    return rdd.groupByKey().flatMap(lambda x: count_new_pairs(x)).sortByKey()

In [ ]:
rdd_iterate_reduce = iterate_reduce_rdd(rdd_iterate_map)
rdd_iterate_reduce.take(20)

In [6]:
def compute_rdd(rdd):
    nb_iteration = 0
    while True:
        nb_iteration += 1
        start_pair = new_pairs.value

        rdd = iterate_map_rdd(rdd)
        rdd = iterate_reduce_rdd(rdd)
        rdd = rdd.distinct()

        print(f"Number of new pairs for iteration #{nb_iteration}:\t{new_pairs.value}")
        if start_pair == new_pairs.value:
            print("\nNo new pair, end of computation")
            break

    return rdd

In [8]:
rdd_raw = load_rdd(dataset_path)
rdd = preprocess_rdd(rdd_raw)

start_time = time.time()
rdd = compute_rdd(rdd)
print(f"Nb of connected components in the graph: {rdd.map(lambda x : x[1]).distinct().count()}")
print(f"Duration in seconds: {time.time() - start_time}")

Number of new pairs for iteration #1:	8
Number of new pairs for iteration #2:	30


Number of new pairs for iteration #3:	47


Number of new pairs for iteration #4:	51


Number of new pairs for iteration #5:	51

No new pair, end of computation
Duration in seconds: 21.188074350357056


Nb of connected components in the graph: 2


In [ ]:
import time
import pyspark

# This function will perform CCF nb_iteration
# It returns the rdd with new pairs calculated by CCF nb_iterations
def Calculate_CCF(rdd):
    nb_iteration = 0

    while True:
        nb_iteration += 1
        start_nb_pairs = new_pairs.value

        rdd_map  = iterate_map_rdd(rdd) # rdd.union(rdd.map(lambda x : (x[1], x[0])))

        # CCF-Iterate REDUCE
        ccf_iterate_reduce = rdd_map.groupByKey().flatMap(lambda x: countnew_pairs(x)).sortByKey()

        # CFF-Dedup MAP & REDUCE
        ccf_dedup_map_reduce = ccf_iterate_reduce.distinct()

        rdd = ccf_dedup_map_reduce
        print("Itération : ", nb_iteration, "Number of new_pairs : ", new_pairs.value)
        
        if start_nb_pairs == new_pairs.value:
            break

    return rdd

In [ ]:
new_pairs = spark_context.accumulator(0)
# dataset = spark_context.textFile(dataset_path, use_unicode="False")
# #graph = prepare_dataset(dataset)
# graph = rdd

rdd_raw = load_rdd(dataset_path)
rdd = preprocess_rdd(rdd_raw)


t1 = time.time()
rdd = Calculate_CCF(rdd)
t2 = time.time()
print("Duration (s) :", t2 - t1)

In [ ]:
graph.count()

In [ ]:
graph.map(lambda x : x[1]).distinct().count()